## Problem Statement

* we will predict the rating for amazon review
* We have only rating scores for product between (1 to 5) and basis this scores we will predict weather the review is +ive or -ive
* Considering 1 and 2 as -ive rating and 3,4,5 as +ive rating
* Datasets has been taken from -https://nijianmo.github.io/amazon/
  1) **Overall** - score of the review (between 1 to 5)

### Install libraries

In [2]:
#!pip install gensim
#!pip install bs4
#!pip install lxml
!pip install html5lib
#nltk.download('wordnet')

### Import Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn libraries
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve,auc
from sklearn  import metrics

#nlp libraries
#import nltk: This line imports the NLTK library into your Python environment. 
# NLTK provides various tools and resources for working with human language data.

# nltk.download('punkt'): 
# The 'punkt' tokenizer is used for tokenization, which means breaking text into individual words or sentences (tokenizination)
import nltk
import re #regular expressions lib
import string
nltk.download('punkt') 
nltk.download('stopwords') #downloading the stopwords from nltk library

from nltk.stem.porter import PorterStemmer  #for stemming
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import gensim; print("Gensim:",gensim.__version__)
from gensim.models import Word2Vec,KeyedVectors

import pickle,os, sys,json, warnings

warnings.filterwarnings("ignore")
%matplotlib inline




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maste\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maste\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Gensim: 4.3.0


### Load Dataset

In [4]:
import gzip
#since dataset is in json.gz file , we will create a function to read the data and  load it into dataframe

def parseJsongz(filepath):
    with gzip.open(filepath,'rb') as f:
        df=pd.read_json(f,lines=True)
    return df
df1= parseJsongz("Dataset/All_Beauty_5.json.gz")
df2= parseJsongz("Dataset/AMAZON_FASHION_5.json.gz")
df3= parseJsongz("Dataset/Appliances_5.json.gz")
df=pd.concat([df1,df2,df3])
df.shape

(10722, 12)

In [5]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"09 1, 2016",A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN
1,5,True,"11 14, 2013",A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN
2,1,True,"08 18, 2013",A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN
3,5,False,"05 3, 2011",A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN
4,5,True,"05 6, 2011",A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10722 entries, 0 to 2276
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   overall         10722 non-null  int64 
 1   verified        10722 non-null  bool  
 2   reviewTime      10722 non-null  object
 3   reviewerID      10722 non-null  object
 4   asin            10722 non-null  object
 5   style           7769 non-null   object
 6   reviewerName    10722 non-null  object
 7   reviewText      10701 non-null  object
 8   summary         10717 non-null  object
 9   unixReviewTime  10722 non-null  int64 
 10  vote            2923 non-null   object
 11  image           1032 non-null   object
dtypes: bool(1), int64(2), object(9)
memory usage: 1015.7+ KB


In [7]:
df['unixTime']=pd.to_datetime(df['unixReviewTime'],unit='s') #parsing the pandas datetime into standard datetime
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,unixTime
0,5,True,"09 1, 2016",A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN,2016-09-01
1,5,True,"11 14, 2013",A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN,2013-11-14
2,1,True,"08 18, 2013",A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN,2013-08-18
3,5,False,"05 3, 2011",A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN,2011-05-03
4,5,True,"05 6, 2011",A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN,2011-05-06


### Data Cleaning

In [8]:
#remove the duplicate reviews, many people submit the same reviews or fake reviews

#sort the datafram acording reviewerName,unixTime,summary,reviewtext

df_sorted=df.sort_values(["reviewerName","unixTime","summary","reviewText"],axis=0,ascending=True,
                         inplace=False,
                         kind='quicksort',
                         na_position='last'
                         )



                          


In [9]:
print(df_sorted.head())
print(df_sorted.shape)

      overall  verified   reviewTime      reviewerID        asin  \
4924        5     False  08 26, 2017  A2MJ8OL2FYN7CW  B00W259T7G   
4187        5     False   09 4, 2017  A2MJ8OL2FYN7CW  B001LNODUS   
5034        5     False   09 4, 2017  A2MJ8OL2FYN7CW  B019FWRG3C   
4856        5     False  09 21, 2017  A2MJ8OL2FYN7CW  B00W259T7G   
4849        5     False  09 27, 2017  A2MJ8OL2FYN7CW  B00W259T7G   

                                                style reviewerName  \
4924   {'Size:': ' 250 Gram', 'Color:': ' Olive Oil'}     Lynne E.   
4187                        {'Color:': ' Shower Gel'}     Lynne E.   
5034                        {'Color:': ' Shower Gel'}     Lynne E.   
4856    {'Size:': ' 150 Gram', 'Color:': ' Sea Salt'}     Lynne E.   
4849  {'Size:': ' 250 Gram', 'Color:': ' Sandalwood'}     Lynne E.   

                                             reviewText  \
4924  This PRE DE PROVENCE SHEA BUTTER ENRICHED ARTI...   
4187  Lavender is my favorite soap fragrance, so it'

In [10]:
#drop duplicates based on reviewerName,reviewText,unixreviewtime,summary

df_uniq= df_sorted.drop_duplicates(subset={'reviewerName','reviewText','summary','unixReviewTime'},keep='first',inplace=False)
df_uniq.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,unixTime
4924,5,False,"08 26, 2017",A2MJ8OL2FYN7CW,B00W259T7G,"{'Size:': ' 250 Gram', 'Color:': ' Olive Oil'}",Lynne E.,This PRE DE PROVENCE SHEA BUTTER ENRICHED ARTI...,"Feels Luxurious, Doesn't Dry Out Sensitive Skin",1503705600,NaN,NaN,2017-08-26
4187,5,False,"09 4, 2017",A2MJ8OL2FYN7CW,B001LNODUS,{'Color:': ' Shower Gel'},Lynne E.,"Lavender is my favorite soap fragrance, so it'...",Clear Gel Creates Nice Lather With Delicate La...,1504483200,NaN,NaN,2017-09-04
4856,5,False,"09 21, 2017",A2MJ8OL2FYN7CW,B00W259T7G,"{'Size:': ' 150 Gram', 'Color:': ' Sea Salt'}",Lynne E.,"I love Pre de Provence soaps, but was intensel...",Luxurious French Soap Has Fresh Sea Breeze Fra...,1505952000,7,NaN,2017-09-21
4849,5,False,"09 27, 2017",A2MJ8OL2FYN7CW,B00W259T7G,"{'Size:': ' 250 Gram', 'Color:': ' Sandalwood'}",Lynne E.,This PRE DE PROVENCE Sandalwood SHEA BUTTER EN...,"Luxurious French Soap With Mild, Masculine Fra...",1506470400,NaN,NaN,2017-09-27
4848,5,False,"09 27, 2017",A2MJ8OL2FYN7CW,B00W259T7G,"{'Size:': ' 250 Gram', 'Color:': ' Patchouli'}",Lynne E.,This PRE DE PROVENCE PATCHOULI SHEA BUTTER ENR...,Luxurious French Soap With Musky Masculine Fra...,1506470400,NaN,NaN,2017-09-27


In [11]:
df_uniq.shape

(1998, 13)

In [12]:
#dropping nan values from reviewText column
df_uniq.dropna(subset=['reviewText'],inplace=True)
df_uniq.isna().sum()

overall              0
verified             0
reviewTime           0
reviewerID           0
asin                 0
style              234
reviewerName         0
reviewText           0
summary              1
unixReviewTime       0
vote              1775
image             1950
unixTime             0
dtype: int64

In [13]:
#dropping the nan values from summary column
df_uniq.dropna(subset=['summary'],inplace=True)
df_uniq.isna().sum()

overall              0
verified             0
reviewTime           0
reviewerID           0
asin                 0
style              234
reviewerName         0
reviewText           0
summary              0
unixReviewTime       0
vote              1774
image             1949
unixTime             0
dtype: int64

In [14]:
#what percentage of total dataset is the df_unique

np.round((df_uniq.shape[0]/df_sorted.shape[0])*100,2)

18.6

##### Display full text


In [15]:
pd.set_option('display.max_colwidth',None)
print(df_uniq['reviewText'].sample(10))

28                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

##### inference - review text contains mainy special characters

## Text pre processing 
* remove the HTML tags and special characters if present
* make words to lower and remove punctuations if any
* remove stopwords and apply stemming or lemmization

In [16]:
#lets remove the html tags and urls , if present

#function to perform text preprocessing

from bs4 import BeautifulSoup
import lxml
import html5lib

lemmatizer=WordNetLemmatizer()


def expand(phrase):
    phrase=re.sub(r"won't","will not",phrase)
    phrase=re.sub(r"can't","can not",phrase)
    phrase=re.sub(r"n\t","not",phrase)
    phrase=re.sub(r"\'re","are",phrase)
    phrase=re.sub(r"\'s","is",phrase)
    phrase=re.sub(r"\'d","would",phrase)
    phrase=re.sub(r"\'ll","will",phrase)
    phrase=re.sub(r"\'t","not",phrase)
    phrase=re.sub(r"\'ve","have",phrase)
    phrase=re.sub(r"\'m","am",phrase)
    return phrase
    

def text_processing(review):
    
    #removes the html tags and url's
    processed_text=[]
    review=re.sub(r"http\S+","",review)
    reviewText =BeautifulSoup(review,features="html.parser").get_text()
    #expand the words to base form form
    reviewText=expand(reviewText)

    #make words into lower letter

    reviewText=reviewText.lower()
    
    #remove numeric and special characters
    reviewText= re.sub("\S*\d\S*","",reviewText).strip() #only alphabet regex
    reviewText=re.sub('[^A-Za-z]+',' ',reviewText)  #special character regex
    
    #tokenization
    reviewText=nltk.word_tokenize(reviewText)

    #removing stopwrds and punctuations

    for i in reviewText:
        if i not in stopwords.words('english') and i not in string.punctuation:
            processed_text.append(i.strip())
    
    reviewText=processed_text[:]
    processed_text.clear()

    #applying stemming or lamitization

    for i in reviewText:
        processed_text.append(lemmatizer.lemmatize(i))


        
    return processed_text
            
    
        
    #print(processed_text)
    

In [17]:
#processing the reviewText data
df_uniq['processedText']=df_uniq['reviewText'].apply(text_processing)

In [18]:
df_uniq['summary'].values[:5]

array(["Feels Luxurious, Doesn't Dry Out Sensitive Skin",
       'Clear Gel Creates Nice Lather With Delicate Lavender Scent',
       'Luxurious French Soap Has Fresh Sea Breeze Fragrance',
       'Luxurious French Soap With Mild, Masculine Fragrance',
       'Luxurious French Soap With Musky Masculine Fragrance'],
      dtype=object)

In [19]:
#processing the summary data
df_uniq['processedSummary']=df_uniq['summary'].apply(text_processing)

In [20]:
df_uniq.head(20)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,unixTime,processedText,processedSummary
4924,5,False,"08 26, 2017",A2MJ8OL2FYN7CW,B00W259T7G,"{'Size:': ' 250 Gram', 'Color:': ' Olive Oil'}",Lynne E.,"This PRE DE PROVENCE SHEA BUTTER ENRICHED ARTISANAL FRENCH SOAP BAR (Olive Oil (250 g)) lathers beautifully, and feels luxurious. It doesn't dry out or irritate my sensitive skin, so I can use it every day. This is a large 250 gram olive oil bar that gives good value for the money (about $7 on Amazon).","Feels Luxurious, Doesn't Dry Out Sensitive Skin",1503705600,NaN,NaN,2017-08-26,"[pre, de, provence, shea, butter, enriched, artisanal, french, soap, bar, olive, oil, g, lather, beautifully, feel, luxurious, doesnnot, dry, irritate, sensitive, skin, use, every, day, large, gram, olive, oil, bar, give, good, value, money, amazon]","[feel, luxurious, doesnnot, dry, sensitive, skin]"
4187,5,False,"09 4, 2017",A2MJ8OL2FYN7CW,B001LNODUS,{'Color:': ' Shower Gel'},Lynne E.,"Lavender is my favorite soap fragrance, so it's no surprise that I love this PRE DE PROVENCE FRENCH LAVENDER BATH & SHOWER GEL. For the shower, a pump or two is enough for a nice lather, and the lather washes off easily.\n\nThe lavender fragrance is delicate, and it lingers for only a short time. It doesn't overwhelm perfume, aftershave lotion, shampoo, or other scented products you may like to use. The clear gel arrives in an attractive square pump dispenser.\n\nThe shower gel leaves my skin silky smooth, and it doesn't irritate my sensitive skin.",Clear Gel Creates Nice Lather With Delicate Lavender Scent,1504483200,NaN,NaN,2017-09-04,"[lavender, favorite, soap, fragrance, itis, surprise, love, pre, de, provence, french, lavender, bath, shower, gel, shower, pump, two, enough, nice, lather, lather, wash, easily, lavender, fragrance, delicate, lingers, short, time, doesnnot, overwhelm, perfume, aftershave, lotion, shampoo, scented, product, may, like, use, clear, gel, arrives, attractive, square, pump, dispenser, shower, gel, leaf, skin, silky, smooth, doesnnot, irritate, sensitive, skin]","[clear, gel, creates, nice, lather, delicate, lavender, scent]"
4856,5,False,"09 21, 2017",A2MJ8OL2FYN7CW,B00W259T7G,"{'Size:': ' 150 Gram', 'Color:': ' Sea Salt'}",Lynne E.,"I love Pre de Provence soaps, but was intensely curious about the Sea Salt bar. What would it smell like? Why would I want to wash my hands with sea salt? Happily, this PRE DE PROVENCE SEA SALT FRENCH SOAP BAR (150g, 5.2 Ounce) is no different from the other Pre de Provence soaps--it smells wonderful, and produces a rich, thick lather.\n\nThe fragrance is hard to describe. Its kind of floral, but maybe is best described as fresh sea breeze. It makes one think of fresh air and sunshine. The fragrance doesn't linger overlong. The soap doesn't irritate my sensitive skin, but may be a little more drying than other Pre de Provence soaps (unless it's my imagination working overtime).\n\nThe soap definitely contains salt. The ingredients are: sodium palmate, sodium palm kernelate, water, fragrance, palm acid, titanium dioxide, glycerin, palm kernel acid, SODIUM CHLORIDE, shea butter, sodium hydroxide, tetrasodium edta, tetrasodium etidronate, benzyl salicylate, linalool, geraniol, citral.",Luxurious French Soap Has Fresh Sea Breeze Fragrance,1505952000,7,NaN,2017-09-21,"[love, pre, de, provence, soap, intensely, curious, sea, salt, bar, would, smell, like, would, want, wash, hand, sea, salt, happily, pre, de, provence, sea, salt, french, soap, bar, ounce, different, pre, de, provence, soap, smell, wonderful, produce, rich, thick, lather, fragrance, hard, describe, kind, floral, maybe, best, described, fresh, sea, breeze, make, one, think, fresh, air, sunshine, fragrance, doesnnot, linger, overlong, soap, doesnnot, irritate, sensitive, skin, may, little, drying, pre, de, provence, soap, unless, itis, imagination, working, overtime, soap, definitely, contains,

#### feature Engineerig for Text 

In [21]:
#save the processed data 
df_uniq.to_csv('processed_text.csv')

In [22]:
#BAg of Words
count_vect=CountVectorizer(lowercase=True)  #in scikit learn
count_vect.fit(df_uniq['processedText'].astype('str'))  #.astype() will convert lost items to strings 
print("Sample:",count_vect.get_feature_names()[:10])  #we can also preprocees to remove words like 'aaa','abd'

Sample: ['aaa', 'abating', 'abbey', 'abd', 'ability', 'abit', 'able', 'abrasing', 'abrasive', 'absence']


In [23]:
final_counts=count_vect.transform(df_uniq['processedText'].astype('str'))
print(f"the type of count vectorizer- {type(final_counts)}")
print(f"the shape of out text of Bag Of Words{final_counts.shape}")


the type of count vectorizer- <class 'scipy.sparse._csr.csr_matrix'>
the shape of out text of Bag Of Words(1994, 5720)


In [24]:
dtm=final_counts.toarray() #documentTermMatrix
print(f"Shape of doc term matrix {dtm.shape}")
dtm[:10,:20]

Shape of doc term matrix (1994, 5720)


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

In [25]:
#implement bigrams and trigrams
#typically we should not remove the stopwords,because there can be combinations formed with stop words.

count_vect_bigram=CountVectorizer(ngram_range=(1,2),min_df=10,max_features=5000) #bigram
final_bigram_counts=count_vect_bigram.fit_transform(df_uniq['processedText'].astype('str'))
print(f"shape of DocTerm Matrix{final_bigram_counts.shape}")
print(count_vect_bigram.get_feature_names()[:200])

shape of DocTerm Matrix(1994, 954)
['able', 'absolutely', 'absolutely love', 'active', 'active ingredient', 'actually', 'add', 'added', 'adult', 'advertised', 'afterwards', 'age', 'ago', 'air', 'alcohol', 'alcohol free', 'allow', 'almost', 'along', 'already', 'also', 'although', 'always', 'amazing', 'amazon', 'amount', 'another', 'anymore', 'anyone', 'anything', 'anyway', 'anywhere', 'apply', 'applying', 'appreciate', 'arch', 'arch support', 'area', 'aroma', 'around', 'arrived', 'ask', 'available', 'average', 'away', 'awesome', 'baby', 'back', 'bad', 'bag', 'bar', 'bar soap', 'base', 'based', 'basic', 'bath', 'bathroom', 'battery', 'beautiful', 'beauty', 'bed', 'believe', 'benefit', 'best', 'best shampoo', 'better', 'big', 'bigger', 'bit', 'black', 'blend', 'blossom', 'blow', 'blue', 'body', 'body lotion', 'body wash', 'bother', 'bottle', 'bottom', 'bought', 'box', 'brace', 'brand', 'break', 'breaking', 'breath', 'bright', 'bring', 'brush', 'brushing', 'bubble', 'burn', 'burning', 'but

In [26]:
count_vect_trigram=CountVectorizer(ngram_range=(1,3),min_df=10,max_features=5000) #trigram
final_bigram_counts=count_vect_trigram.fit_transform(df_uniq['processedText'].astype('str'))
print(f"shape of DocTerm Matrix{final_bigram_counts.shape}")
print(count_vect_trigram.get_feature_names()[:200])

shape of DocTerm Matrix(1994, 961)
['able', 'absolutely', 'absolutely love', 'active', 'active ingredient', 'actually', 'add', 'added', 'adult', 'advertised', 'afterwards', 'age', 'ago', 'air', 'alcohol', 'alcohol free', 'allow', 'almost', 'along', 'already', 'also', 'although', 'always', 'amazing', 'amazon', 'amount', 'another', 'anymore', 'anyone', 'anything', 'anyway', 'anywhere', 'apply', 'applying', 'appreciate', 'arch', 'arch support', 'area', 'aroma', 'around', 'arrived', 'ask', 'available', 'average', 'away', 'awesome', 'baby', 'back', 'bad', 'bag', 'bar', 'bar soap', 'base', 'based', 'basic', 'bath', 'bathroom', 'battery', 'beautiful', 'beauty', 'bed', 'believe', 'benefit', 'best', 'best shampoo', 'better', 'big', 'bigger', 'bit', 'black', 'blend', 'blossom', 'blow', 'blue', 'body', 'body lotion', 'body wash', 'bother', 'bottle', 'bottom', 'bought', 'box', 'brace', 'brand', 'break', 'breaking', 'breath', 'bright', 'bring', 'brush', 'brushing', 'bubble', 'burn', 'burning', 'but

In [27]:
#TF-IDF
tf_idf_vect=TfidfVectorizer(ngram_range=(1,2),min_df=10)
tf=tf_idf_vect.fit_transform(df_uniq['processedText'].astype('str')).toarray()
print(f"some sample features, unique words in the corpus-{tf_idf_vect.get_feature_names()[0:10]}")




some sample features, unique words in the corpus-['able', 'absolutely', 'absolutely love', 'active', 'active ingredient', 'actually', 'add', 'added', 'adult', 'advertised']


In [28]:
print(f"the no of unique words including bigram and trigram are - {len(tf_idf_vect.get_feature_names())}")

the no of unique words including bigram and trigram are - 954


In [29]:
tf=pd.DataFrame(tf,columns=tf_idf_vect.get_feature_names())  #storing tf-idf of words into a dataframe
tf.shape 

(1994, 954)

### inference- now we have got our reviews converted into vectors , which are nithing ut the features(X) lets build a logistic regression model on this data to predict rating 

In [30]:
df_uniq['overall'].value_counts()

overall
5    1534
4     232
3     124
1      55
2      49
Name: count, dtype: int64

In [31]:
#each class ratio 
(df_uniq['overall'].value_counts()/df_uniq['overall'].shape[0])*100


overall
5    76.930792
4    11.634905
3     6.218656
1     2.758275
2     2.457372
Name: count, dtype: float64

##### inference - the target variable is imbalanced so we will use stratified k-fold validation 

## Model building  

In [32]:
#lets import logistic regression packages

from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score,roc_auc_score,roc_curve,classification_report
import time


In [38]:
print(X.shape,y.shape)

(1994, 954) (1994,)


In [47]:
X=tf
y=df_uniq['overall']
y=y.reset_index(drop=True)
df_tf=pd.concat([X,y.rename('y')],axis=1)
df_tf.shape

(1994, 955)

In [48]:
# we will use multiple models to check whihc model works best to predict the rating
#lets create  two dataframe to store hyperparameter values and models metrics
df_hyperparameter=pd.DataFrame(columns=['modelName','hyperparameter'])
df_models_output= pd.DataFrame(columns=['model','accuracy','precision','recall','F1Score'])
df_hyperparameter

,modelName,hyperparameter


In [49]:
#lets create a function to evaluate the models output, since this multiclass problem roc curve calculation will eb different

def compute_eval_metric(X_test,y_actual,y_pred,y_predProb,y_pred_encoded,lb):
    accu=accuracy_score(y_actual,y_pred)
    print(f" accuracy is -{accu}")
    cm=confusion_matrix(y_actual,y_pred,labels=y.value_counts().index)
    print(f" confusion matrix is \n {cm}")
    print("\n")
    for i in y.value_counts().index:
        print(f" class {i}")
        print(f" TP- {sum([1 if ((m==i) & (n==i)) else 0 for (m,n) in zip(y_actual,y_pred)])} ")
        print(f" TN- {sum([1 if ((m!=i) & (n!=i)) else 0 for (m,n) in zip(y_actual,y_pred)])} ")
        print(f" FP- {sum([1 if ((m!=i) & (n==i)) else 0 for (m,n) in zip(y_actual,y_pred)])} ")
        print(f" FN- {sum([1 if ((m==i) & (n!=i)) else 0 for (m,n) in zip(y_actual,y_pred)])} ")    
        TP=np.diag(cm).astype(float)
        FN=cm.sum(axis=1)-np.diag(cm)
        FP=cm.sum(axis=0)-np.diag(cm)
        TN=cm.sum()-(TP+FP+FN)
        print(f" TPR/Sensitivity/Recall -{np.mean(TP/(TP+FN))}")
        print(f" TNR -{np.mean(TN/(TN+FP))}")
        
        classid=np.flatnonzero(lb.classes_==i)[0] #this code retrieves the column index value from labelencoded columns
        #means corresponding target class encoded to which column index in labelencoder
               
        roc=roc_auc_score(y_pred_encoded[:,classid],y_pred_prob[:,classid],multi_class='ovr',average='macro')
        #here above we are slicing only values equivalent to each class to compute ROC
        print(f"roc-{roc}")
        print(f"")
    print(f"\n classification report :- \n {classification_report(y_actual,y_pred)}")

    
    
    

In [50]:
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=2024)
df_tf['stratify']=df_tf['y'].astype(str)


#this will split the entire dataframe in to 5 folds , where each fold will have equal share of classes

for fold, (train_index,test_index) in enumerate(skf.split(X,df_tf['stratify'])):
    df_tf.loc[test_index,'Kfold']=fold  #this will add a column with fold numbers as value
    X_train,X_test=X.loc[train_index],X.loc[test_index]
    y_train,y_test=y.loc[train_index],y.loc[test_index]
    
    print(f"Fold:{fold},train set :{len(train_index)},test set :{len(test_index)}")
    
    


#function to get each fold data from above stratifiedKfold

def get_startified_datasets(fold):
    #passed fold parameter will be treated as test rows
    train_df=df_tf[df_tf['Kfold']!=fold].sample(frac=1) #get training fold data
    test_df= df_tf[df_tf['Kfold']==fold].sample(frac=1) #get validation fold data
    X_train=train_df.drop(columns=['y','Kfold'])
    y_train=train_df['y']
    X_test=test_df.drop(columns=['y','Kfold'])
    y_test=test_df['y']
    return (X_train,X_test,y_train,y_test)

    

Fold:0,train set :1595,test set :399
Fold:1,train set :1595,test set :399
Fold:2,train set :1595,test set :399
Fold:3,train set :1595,test set :399
Fold:4,train set :1596,test set :398


### Logistic regression model 

In [51]:
# %%time

# #hyperparameter tuning using gridsearchCV
# lr=LogisticRegression(multi_class='multinomial')
# params={"C":[0.00001,0.0001,0.0005,.001,0.005,.01,.1],"solver":['lbfgs'],"penalty":["none","l1","l2","elasticnet"]}
# gs_lr=GridSearchCV(estimator=lr,param_grid=params,verbose=1,scoring=accuracy_score,cv=skf)
# gs_lr.fit(X_train,y_train)
# print(gs_lr.best_estimator_)
# print(gs_lr.best_score_)

In [52]:
df_hyperparameter=pd.concat([df_hyperparameter,pd.DataFrame([{'modelName':'LogisticRegression','hyperparameter':'C=1e-05, multi_class=multinomial, penalty=none'}])],ignore_index=True)
df_hyperparameter

,modelName,hyperparameter
0,LogisticRegression,"C=1e-05, multi_class=multinomial, penalty=none"


In [53]:
from sklearn.preprocessing import LabelBinarizer #use to conver multiclass labels in dense binary matrix , helpful to get ROC_AUC_CURVE


for fold in [0,1,2]:
    lr=LogisticRegression(C=.00005,multi_class='multinomial',penalty='none')
    X_train,X_test,y_train,y_test=get_startified_datasets(fold)
    print(f"Fold {fold} data for Logistic regression")
    lr.fit(X_train,y_train)
    y_pred=lr.predict(X_test)  
    
    y_pred_prob=lr.predict_proba(X_test)
    
    #label encoded target columns
    lb=LabelBinarizer().fit(y_train)
    y_pred_encoded=lb.fit_transform(y_test)
    
    
    compute_eval_metric(X_test,y_test,y_pred,y_pred_prob,y_pred_encoded,lb)
    print("\n")
    
    


Fold 0 data for Logistic regression
 accuracy is -0.9924812030075187
 confusion matrix is 
 [[307   0   0   0   0]
 [  0  47   0   0   0]
 [  0   0  24   0   0]
 [  0   0   0  11   0]
 [  0   0   3   0   7]]


 class 5
 TP- 307 
 TN- 92 
 FP- 0 
 FN- 0 
 TPR/Sensitivity/Recall -0.9400000000000001
 TNR -0.9984
roc-1.0

 class 4
 TP- 47 
 TN- 352 
 FP- 0 
 FN- 0 
 TPR/Sensitivity/Recall -0.9400000000000001
 TNR -0.9984
roc-1.0

 class 3
 TP- 24 
 TN- 372 
 FP- 3 
 FN- 0 
 TPR/Sensitivity/Recall -0.9400000000000001
 TNR -0.9984
roc-1.0

 class 1
 TP- 11 
 TN- 388 
 FP- 0 
 FN- 0 
 TPR/Sensitivity/Recall -0.9400000000000001
 TNR -0.9984
roc-1.0

 class 2
 TP- 7 
 TN- 389 
 FP- 0 
 FN- 3 
 TPR/Sensitivity/Recall -0.9400000000000001
 TNR -0.9984
roc-1.0


 classification report :- 
               precision    recall  f1-score   support

           1       1.00      1.00      1.00        11
           2       1.00      0.70      0.82        10
           3       0.89      1.00      0.94      

### Inference - Model was able to predict all rating classes correctly  

## Model building for the dataset created using Average word2vec model 

In [54]:
df_uniq[df_uniq['processedText'].apply(lambda x : len(x)==0)].index  #having empty list values in processedText column
df_uniq.drop(index=df_uniq[df_uniq['processedText'].apply(lambda x : len(x)==0)].index,inplace=True)

In [55]:
#wordToVec - creating one single list with all reviews
i=0
list_of_review=[]
for i,review in enumerate(df_uniq['processedText']):
    if len(review)==0:
        print(i)
    
    
    list_of_review.append(review)
len(list_of_review)


1987

In [56]:
ram_gt_16g=True
#change below value to true/false to use resepective model
use_google_w2v=True
train_w2v=False

if train_w2v: #train Word2Vec
    #only words which occure 5 or more times.
    w2v_model=Word2Vec(list_of_review,min_count=5,vector_size=50,workers=4)
    print(w2v_model.wv.most_similar('buy'))
    print("\n","-"*50,"\n")
    print(w2v_model.wv.most_similar('bad'))
elif use_google_w2v and ram_gt_16g:
    
    #use pretrained wordToVec model and load it
    if os.path.isfile('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin',binary=True)
        #print(w2v_model.wv.most_similar('great'))
        #print(w2v_model.wv.most_similar('bad'))
        print(w2v_model.most_similar('great'))
    else:
        print("word2Vec file not found,set train_w2v =True to train your model")
    
    

[('terrific', 0.7989332675933838), ('fantastic', 0.7935210466384888), ('tremendous', 0.7748856544494629), ('wonderful', 0.7647868394851685), ('good', 0.7291509509086609), ('incredible', 0.7032873630523682), ('marvelous', 0.6971103549003601), ('phenomenal', 0.6841565370559692), ('amazing', 0.6634128093719482), ('awesome', 0.6510507464408875)]


In [57]:
w2v_words=list(w2v_model.index_to_key)
print("number of words",len(w2v_words))
print("sample words-occured at least 5 times:\n",w2v_words[0:50])

number of words 3000000
sample words-occured at least 5 times:
 ['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said', 'was', 'the', 'at', 'not', 'as', 'it', 'be', 'from', 'by', 'are', 'I', 'have', 'he', 'will', 'has', '####', 'his', 'an', 'this', 'or', 'their', 'who', 'they', 'but', '$', 'had', 'year', 'were', 'we', 'more', '###', 'up', 'been', 'you', 'its', 'one', 'about', 'would', 'which', 'out']


In [58]:
#avraging  Word2Vec- taking average of vectors for each list of review words, each list of vectors are averaged

#converting entire review to vector unlike converting each word to vector


from tqdm  import tqdm

review_vectors=[]
words_not_present_in_word2vec_dictionary=[]
empty_review=[]
print(len(list_of_review))
for review in tqdm(list_of_review): #each lst in list of reviews
    review_vec=np.zeros(300) #set it to 0 intially , 50 because vector dimesion is 50, use 300 for google's word2Vec model
    cnt_words=0; #num of words with a valid vector in the sentence/review
    #print(review)
    for word in review:
        #print(f"list {len(review_vectors)}")
        if word in w2v_words:
            wordVector=w2v_model[word] #get the vector for word
            review_vec +=wordVector
            cnt_words+=1
        else:
            #print("word not present")
            words_not_present_in_word2vec_dictionary.append(word)
    if cnt_words != 0:
        review_vec /=cnt_words
        review_vectors.append(review_vec)
    else:
        empty_review.append(len(review_vectors))
        #print(review_vectors)
print(len(review_vectors))
#print((review_vectors[0]))
    
    




1987


100%|███████████████████████████████████████████████████████████████████████████████████████████| 1987/1987 [02:14<00:00, 14.76it/s]

1986
[-3.23252958e-02  3.00939223e-02 -1.06461469e-02  1.03159287e-01
 -3.19421993e-02  3.45055075e-02  2.44194480e-02 -1.53952206e-01
  3.08299345e-02  1.15835751e-01 -7.57589901e-02 -1.29437615e-01
 -4.01368983e-02  4.45866304e-02 -1.02675494e-01  1.46086749e-01
 -3.21152631e-02  1.50003882e-01 -2.10382798e-02 -3.29666138e-02
 -1.93750718e-02 -2.26632287e-02  1.06219123e-02  3.12859030e-02
  2.10966222e-02  3.03847369e-02 -5.37156498e-02  1.10651353e-01
 -2.16351677e-02 -3.05463006e-02 -1.79420920e-02  2.89997774e-02
  2.00267119e-02 -4.88640280e-03 -6.82525635e-02 -4.44161471e-02
  3.16036449e-02  5.74044620e-03  4.19571820e-02  1.99728573e-02
  8.72766831e-02 -2.16300067e-02  1.30482393e-01  3.85415694e-02
 -7.87678887e-03 -9.51986874e-02 -1.41732609e-01  4.86401951e-02
  3.25371237e-03 -6.53435202e-04 -1.14743850e-02  6.36675218e-02
 -2.98533720e-03 -4.32155834e-02 -2.65924790e-02  2.88115109e-02
 -3.54462231e-02 -8.50686466e-02 -3.22086110e-02 -1.12205281e-01
 -3.03290872e-02  2.

In [59]:
for review in list_of_review[:5]:
    print(review)

['pre', 'de', 'provence', 'shea', 'butter', 'enriched', 'artisanal', 'french', 'soap', 'bar', 'olive', 'oil', 'g', 'lather', 'beautifully', 'feel', 'luxurious', 'doesnnot', 'dry', 'irritate', 'sensitive', 'skin', 'use', 'every', 'day', 'large', 'gram', 'olive', 'oil', 'bar', 'give', 'good', 'value', 'money', 'amazon']
['lavender', 'favorite', 'soap', 'fragrance', 'itis', 'surprise', 'love', 'pre', 'de', 'provence', 'french', 'lavender', 'bath', 'shower', 'gel', 'shower', 'pump', 'two', 'enough', 'nice', 'lather', 'lather', 'wash', 'easily', 'lavender', 'fragrance', 'delicate', 'lingers', 'short', 'time', 'doesnnot', 'overwhelm', 'perfume', 'aftershave', 'lotion', 'shampoo', 'scented', 'product', 'may', 'like', 'use', 'clear', 'gel', 'arrives', 'attractive', 'square', 'pump', 'dispenser', 'shower', 'gel', 'leaf', 'skin', 'silky', 'smooth', 'doesnnot', 'irritate', 'sensitive', 'skin']
['love', 'pre', 'de', 'provence', 'soap', 'intensely', 'curious', 'sea', 'salt', 'bar', 'would', 'smell'

In [60]:
len(words_not_present_in_word2vec_dictionary)  #words that are in reviews but not present in word2vec model
len(empty_review) # reviews that are converted to vectors because no review words present in word2vec model 

1

In [61]:
#print output
for i,(review,vec) in enumerate(zip(list_of_review,review_vectors)):
    if i<3:
        print("-"*10,"\n",review,"\n",vec)
    

---------- 
 ['pre', 'de', 'provence', 'shea', 'butter', 'enriched', 'artisanal', 'french', 'soap', 'bar', 'olive', 'oil', 'g', 'lather', 'beautifully', 'feel', 'luxurious', 'doesnnot', 'dry', 'irritate', 'sensitive', 'skin', 'use', 'every', 'day', 'large', 'gram', 'olive', 'oil', 'bar', 'give', 'good', 'value', 'money', 'amazon'] 
 [-3.23252958e-02  3.00939223e-02 -1.06461469e-02  1.03159287e-01
 -3.19421993e-02  3.45055075e-02  2.44194480e-02 -1.53952206e-01
  3.08299345e-02  1.15835751e-01 -7.57589901e-02 -1.29437615e-01
 -4.01368983e-02  4.45866304e-02 -1.02675494e-01  1.46086749e-01
 -3.21152631e-02  1.50003882e-01 -2.10382798e-02 -3.29666138e-02
 -1.93750718e-02 -2.26632287e-02  1.06219123e-02  3.12859030e-02
  2.10966222e-02  3.03847369e-02 -5.37156498e-02  1.10651353e-01
 -2.16351677e-02 -3.05463006e-02 -1.79420920e-02  2.89997774e-02
  2.00267119e-02 -4.88640280e-03 -6.82525635e-02 -4.44161471e-02
  3.16036449e-02  5.74044620e-03  4.19571820e-02  1.99728573e-02
  8.72766831e-0

In [62]:
y=df_uniq['overall'].drop(index=empty_review)  #drop the row where review is empty or not able to get converted to vectors
y.shape

(1986,)

overall
5    1526
4     232
3     124
1      55
2      49
Name: count, dtype: int64

In [208]:
#split the dataset into train and test
X_train,X_test,y_train,y_test=train_test_split(review_vectors,y,test_size=0.3,random_state=2024)

In [209]:
#lest use all common classification models to predict the rating
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

In [210]:
#minmax scaling
ms=MinMaxScaler()
ms.fit(X_train)
X_train=ms.transform(X_train)
X_test=ms.transform(X_test)

### hyperparmater for NB 

In [211]:

#y_train=ms.fit_transform(y_train)
# nb=MultinomialNB()
# params={"alpha":[.0000001,0.00001,.0001,.001,.01,1,5,10,100]}
# rskf=RepeatedStratifiedKFold(n_repeats=5,n_splits=10,random_state=2024)
# gs_nb=GridSearchCV(estimator=nb,param_grid=params,cv=rskf,scoring=f1_score)
# gs_nb.fit(X_train,y_train)
# print(gs_nb.best_estimator_)
# df_hyperparameter=pd.concat([df_hyperparameter,pd.DataFrame([{'modelName':'Multinomial NB','hyperparameter':'C=1e-07'}])],ignore_index=True)
# df_hyperparameter

### Hyperparamter tuning for SVM 

In [212]:
# %%time
# svm=SVC()
# params={"C":[0.000001,.00001,1,100,1000,10000],"gamma":[100,10,0.1,.001],"kernel":['rbf','sigmoid']}
# gs_svm=GridSearchCV(param_grid=params,cv=rskf,estimator=svm,verbose=1)
# gs_svm.fit(X_train,y_train)
# gs_svm.best_estimator_

In [213]:
df_hyperparameter=pd.concat([df_hyperparameter,pd.DataFrame([{'modelName':'SVC','hyperparameter':'C=100,gamma=10'}])],ignore_index=True)

In [214]:
# rf=RandomForestClassifier()
# params={"criterion":['gini','entropy'],"max_features":['sqrt','log2']}
# gs_rf=GridSearchCV(estimator=rf,param_grid=params,verbose=1,cv=rskf,scoring=f1_score,n_jobs=-1)
# gs_rf.fit(X_train,y_train)
# gs_rf.best_params_

In [215]:
gs_rf.best_params_



{'criterion': 'gini', 'max_features': 'sqrt'}

In [216]:
df_hyperparameter=pd.concat([df_hyperparameter,pd.DataFrame([{'modelName':'SVC','hyperparameter':'criterion: gini, max_features: sqrt'}])],ignore_index=True)

### Hyeprparameter tuning for XGbooster 

In [217]:
# %%time
# from xgboost import XGBClassifier
# from sklearn.preprocessing import LabelEncoder
# y_train=LabelEncoder().fit_transform(y_train) #XGBooster model expext y to  start from 0 , thats why using label encoder
# XGB=XGBClassifier()
# #y_train=
# params={"n_estimators":[50,100,150],"learning_rate":[0.001,0.05,.1]}
# gs_xgb=GridSearchCV(estimator=XGB,param_grid=params)
# gs_xgb.fit(X_train,y_train)

In [218]:
gs_xgb.best_params_

{'learning_rate': 0.05, 'n_estimators': 100}

In [219]:
df_hyperparameter=pd.concat([df_hyperparameter,pd.DataFrame([{'modelName':'XGBoost','hyperparameter':'learning_rate: 0.05, n_estimators: 100'}])],ignore_index=True)

### Hyperparameter tuning for AdaBoost 

In [220]:
# %%time
# from sklearn.ensemble import AdaBoostClassifier
# Aboost=AdaBoostClassifier()
# params={"n_estimators":[100,200,300],"learning_rate":[0.001,0.1,1]}
# gs_aboost=GridSearchCV(estimator=Aboost,param_grid=params,cv=rskf,scoring=accuracy_score)
# gs_aboost.fit(X_train,y_train)



In [221]:
gs_aboost.best_params_


{'learning_rate': 0.001, 'n_estimators': 100}

In [222]:
df_hyperparameter=pd.concat([df_hyperparameter,pd.DataFrame([{'modelName':'Adaboost','hyperparameter':'learning_rate: 0.001, n_estimators: 100'}])],ignore_index=True)

In [223]:
#buidling all models
lr=LogisticRegression(C=.00005,multi_class='multinomial',penalty='none') 
nb=MultinomialNB(alpha=0.000001) #multinomial naive bayes
svm=SVC(C=100,gamma=10,probability=True)
rf=RandomForestClassifier(criterion='gini',max_features='sqrt')
xgb=XGBClassifier(learning_rate=0.05,n_estimators=100)
adbst=AdaBoostClassifier(learning_rate=0.001,n_estimators=100)

In [224]:
df_models_output


,model,accuracy,precision,recall,F1Score


In [225]:
lb=LabelBinarizer().fit(y_train)
y_pred_encoded=lb.transform(y_test)

    

In [226]:
y_pred_encoded

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])

In [227]:
lb=LabelBinarizer().fit(y_train)
y_pred_encoded=lb.fit_transform(y_test)


In [229]:
y_test

686     5
497     5
783     5
837     5
2182    4
       ..
4168    4
477     5
252     5
347     4
850     5
Name: overall, Length: 596, dtype: int64

In [232]:
%%time
#fit the train data of each model and save the output to dataframe


for model in [lr,nb,svm,rf,adbst]:
    print(type(model).__name__)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print(y_pred[:1])
    y_pred_prob=model.predict_proba(X_test)
    
    #df_models_output.loc[len(df_models_output)]=compute_eval_metric
    compute_eval_metric(X_test,y_test,y_pred,y_pred_prob,y_pred_encoded,lb)
    

LogisticRegression
[5]
 accuracy is -0.697986577181208
 confusion matrix is 
 [[393  35   8   4   5]
 [ 58   9   2   1   0]
 [ 33   2   8   2   1]
 [ 10   2   2   5   3]
 [  8   1   1   2   1]]


 class 5
 TP- 393 
 TN- 42 
 FP- 109 
 FN- 52 
 TPR/Sensitivity/Recall -0.29796526873224477
 TNR -0.8294693738093709
roc-0.707671701763524

 class 4
 TP- 9 
 TN- 486 
 FP- 40 
 FN- 61 
 TPR/Sensitivity/Recall -0.29796526873224477
 TNR -0.8294693738093709
roc-0.6006110809342748

 class 3
 TP- 8 
 TN- 537 
 FP- 13 
 FN- 38 
 TPR/Sensitivity/Recall -0.29796526873224477
 TNR -0.8294693738093709
roc-0.7094664031620553

 class 1
 TP- 5 
 TN- 565 
 FP- 9 
 FN- 17 
 TPR/Sensitivity/Recall -0.29796526873224477
 TNR -0.8294693738093709
roc-0.8902439024390244

 class 2
 TP- 1 
 TN- 574 
 FP- 9 
 FN- 12 
 TPR/Sensitivity/Recall -0.29796526873224477
 TNR -0.8294693738093709
roc-0.6754189207019395


 classification report :- 
               precision    recall  f1-score   support

           1       0.36   